In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
library(dplyr)
library(devtools)
library(stats)
library(class)
library(ggplot2)
library(lubridate)
library(MESS)
library(rcompanion)
library(lsr)
library(dplyr)
library(glmnet)
library(caret)
library(gridExtra)
library(grid)
library(ggplot2)
library(lattice)
library(tidyverse)
library(data.table)
library(EnvStats)
library(stringr)
#### Installing and loading required libraries
if(!require(fpc)) install.packages("fpc")
if(!require(dbscan)) install.packages("dbscan")
if(!require(devtools)) install.packages("devtools")
#devtools::install_github("kassambara/factoextra")
if(!require(factoextra)) install.packages("factoextra")
library(mclust)
if(!require(data.table)) install.packages("data.table")

In [4]:
Items1 <- read.csv("../input/items5/items_ff5.csv")
head(Items1,1)

In [5]:
Items <- subset(Items1,select=-c(item_id,db_id,asin,title,description,num_of_sales_per_item,item_sell_rank_by_category))
#Items1 <- subset(Items1,select=-c(item_id,db_id,asin,title,description,num_of_sales_per_item,item_sell_rank_by_category))

library(data.table)
for (n in names(Items)) {
    if (n %like% "num_of_sales") {
        Items[[n]][which(Items[[n]]>0)] <- 1
        # rename col name to reflect exact meaning of data
        names(Items)[names(Items)==n] <- str_sub(names(Items[n]),8,-1)
    }
}

Items$location <- as.factor(Items$location)
Items$condition <- as.factor(Items$condition)
Items$success <- as.factor(Items$success)
Items$season <- as.factor(Items$season)
Items$sales_cyber_monday <- as.factor(Items$sales_cyber_monday)
Items$sales_black_friday <- as.factor(Items$sales_black_friday)
Items$sales_colombus_day <- as.factor(Items$sales_colombus_day)
Items$sales_labor_day <- as.factor(Items$sales_labor_day)
Items$sales_presidents_day <- as.factor(Items$sales_presidents_day)
Items$sales_4th_july <- as.factor(Items$sales_4th_july)
Items$sales_newyear <- as.factor(Items$sales_newyear)
Items$sales_xms <- as.factor(Items$sales_xms)
Items$reviews_cnt_category <- as.factor(Items$reviews_cnt_category)
Items$brand_size <- as.factor(Items$brand_size)
Items$popular_brand <- as.factor(Items$popular_brand)
Items$creation_time <- as.Date(Items$creation_time,"%d/%m/%Y")
Items$success <- ifelse(Items$success=='t',1,0)

### characters
strlst <- NULL
numlst <- NULL
for (v in names(Items)) {
        if (v != 'success') {
            if(is.factor(Items[[v]])) {
                strlst <- c(strlst,v)}
            if(is.numeric(Items[[v]])) {numlst <- c(numlst, v)}
            }
}
strlst
numlst

In [13]:
dim(Items)
summary(Items)

# Univariate analisys

In [6]:
initial_EDA <- function(var) {
    results <- c(summary(var), IQR = iqr(x = var), Unique_Count=length(unique(var)),
                count_NA= sum(is.na(var)), percent_NA=sum(is.na(var))/length(var))
    return(results)
}

# Variable: creation time

This variable indicates the time when the item was uploaded for sale in the system. The sale could happen within 30 days (= "success YES") or later or not at all (= "success NO). Due to Database design, I don't have data of actual sale time (if sale occured), therefore this variable isn't significant enough.

In [15]:
summary(Items$creation_time)
#focus on years 2018-2020
length(Items$creation_time[which(Items$creation_time<"2018-01-01")])
Items2 <- Items[which(Items$creation_time >= "2018-01-01"),]
length(Items$creation_time)

In [16]:
ggplot(Items)+
geom_histogram(aes(creation_time),bins=100)

In [17]:
plot(table(month(Items2$creation_time)),ylab="total",xlab="Month of Item sale",main="Creation of Items sale proposal by month")
abline(h=mean(table(month(Items2$creation_time))),col="red")

In [18]:
plot(table(day(Items2$creation_time)),ylab="total",xlab="Month of Item sale",main="Creation of Items sale proposal by month")
abline(h=mean(table(day(Items2$creation_time))),col="red")

In [19]:
table(weekdays(Items2$creation_time))
plot(table(weekdays(Items2$creation_time)),ylab="total",xlab="Month of Item sale",main="Creation of Items sale proposal by month")

In [20]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,5,3,3),mfrow=c(1,4))
for (n in numlst) {
    suppressWarnings(
        if (n != 'creation_time') {
                plot(y= Items$creation_time, x=Items[[n]], ylab=n, xlab='creation_time')
                main=paste('creation_time vs. ',n)
            }
        )
    }

In [21]:
#ggplot(Items)+geom_smooth(aes(x=creation_time,y=returns_cnt),method='loess')
ggplot(data=Items, 
       aes(x = creation_time, 
           y = category_size_rank)) + 
           geom_point(alpha =0.5)+ 
           #facet_wrap(~party) +
  ylab("category_size_rank") +
  xlab("creation_time") +
  scale_x_continuous(breaks = c(0, 5, 10)) + #, limits = c(0,10)) +
  geom_smooth(method=lm,  color = "black", fill="lightgray") +
  geom_smooth(span =  0.3) +
  theme_light()

# Variable: Description Word count

In [22]:
initial_EDA(Items$description_word_cnt)
length(which(Items$description_word_cnt > 33)) # count values higher than Q3

In [23]:
library(ggplot2)
options(repr.plot.width = 5, repr.plot.height = 5)
ggplot(data=Items) +
   geom_density(aes(sqrt(description_word_cnt)))

hist(Items$description_word_cnt[which(Items$description_word_cnt<100)])

In [24]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,5,3,3),mfrow=c(1,4))
for (n in numlst) {
    suppressWarnings(
        if (n != 'description_word_cnt') {
                plot(y= Items$description_word_cnt, x=Items[[n]], xlab=n, ylab='description_word_cnt',
                main=paste('description_word_cnt vs. ',n))
            }
        )
    }

In [25]:
options(repr.plot.width = 20, repr.plot.height = 5)
par(mar=c(3,5,3,3),mfrow=c(1,4))

plot(y= sqrt(Items$description_word_cnt), x=Items$ebay_price, xlab="ebay_price", ylab='description_word_cnt',
                main=paste('description_word_cnt vs. ebay_price'),xlim=c(0,1000))

plot(y= sqrt(Items$description_word_cnt), x=Items$amazon_price, xlab="amazon_price", ylab='description_word_cnt',
                main=paste('description_word_cnt vs. amazon_price'),xlim=c(0,1000))
plot(y= sqrt(Items$description_word_cnt), x=Items$review_count, xlab="review_count", ylab='description_word_cnt',
                main=paste('description_word_cnt vs. review_count'),xlim=c(0,1000))
plot(y= sqrt(Items$description_word_cnt), x=Items$returns_cnt, xlab="returns_cnt", ylab='description_word_cnt',
                main=paste('description_word_cnt vs. returns_cnt'),xlim=c(0,15))
plot(y= sqrt(Items$description_word_cnt), x=Items$profit, xlab="profit", ylab='description_word_cnt',
                main=paste('description_word_cnt vs. profit'),xlim=c(0,200))

we see correlation with ebay_price,amazon price, profit, returns count

In [26]:
Items_t <- filter(Items,description_word_cnt<=33)
#summary(Items_t)
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))

for (n in strlst) {
    suppressWarnings(
        if (n != 'brand') {
        boxplot(description_word_cnt ~ Items_t[[n]], data = Items_t, xlab=n, main=paste('returns count vs. ',n))
            }
        )
    }

We see association with condition,popular brand,brand size,sale in labor day

# Variable: Returns count

In [27]:
initial_EDA(Items$returns_cnt)
table(Items$returns_cnt)
#shapiro.test(Items$returns_cnt)

In [28]:
summary(Items$returns_cnt)
460446/length(Items$returns_cnt)
options(repr.plot.width = 10, repr.plot.height = 5)
par(mar=c(3,3,3,3),mfrow=c(1,2))
hist(Items$returns_cnt)
hist(Items$returns_cnt, xlim= c(0,10), ylim = c(0,30000),breaks = 200)

In [29]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in numlst) {
    suppressWarnings(
        if (n != 'returns_cnt') {
                plot(y= Items$returns_cnt, x=Items[[n]], xlab=n, ylab='returns_cnt',
                main=paste('returns_cnt vs. ',n))
            }
        )
    }

In [30]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,3))
plot(y= Items$returns_cnt, x=Items$review_count, xlab="review_count", ylab='returns_cnt',
                main=paste('returns_cnt vs. review_count'),xlim=c(0,1000))
plot(y= Items$returns_cnt, x=Items$ebay_price, xlab="ebay_price", ylab='returns_cnt',
                main=paste('returns_cnt vs. ebay_price'),xlim=c(0,400))
plot(y= Items$returns_cnt, x=Items$profit, xlab="profit", ylab='returns_cnt',
                main=paste('returns_cnt vs. profit'),xlim=c(0,150))

In [31]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in strlst) {
    suppressWarnings(
        if (n != 'brand') {
        boxplot(returns_cnt ~ Items[[n]], data = Items, xlab=n, main=paste('returns count vs. ',n))
            }
        )
    }

Variable returns count is correlated ebay_price, amazon price, category size ranking and to star_rating

In [32]:
df <- Items %>% 
    group_by(returns_cnt) %>% 
    summarise(rating_avg=mean(star_rating,na.rm=T), rating_sd=sd(star_rating,na.rm=T),
    revsied_rating= mean(revised_star_rating,na.rm=T),n=n()) %>%
    arrange(desc(n))
df

In [33]:
scatter.smooth(Items %>% 
    group_by(returns_cnt) %>% 
    summarise(rating_avg=mean(star_rating,na.rm=T), rating_sd=sd(star_rating,na.rm=T),n=n()) %>%
    arrange(desc(n)),main='rating vs return count')
scatter.smooth(Items %>% 
    group_by(returns_cnt) %>% 
    summarise(revsied_rating= mean(revised_star_rating,na.rm=T),n=n()) %>%
    arrange(desc(n)),main='revised rating vs return count')

we can see the average rating decreases as returns count increase up to 10 and then starts to rise.
When looking at revised rating we see increase in rating as returns increase

# Variable : review_count

In [34]:
initial_EDA(Items$review_count)
# percent of values > Q3
length(na.omit(which(Items$review_count > quantile(Items$review_count, probs=0.75,,na.rm=T))))/
    length(na.omit(Items$review_count,,na.rm=T))*100

In [35]:
sum(is.na(Items$review_count))/length(Items$review_count)
Items %>% filter(review_count==0) %>% tally()

56% NA in review count

In [36]:
options(repr.plot.width = 5, repr.plot.height = 5)
ggplot(data=Items) +
    geom_histogram(aes(review_count))+
    labs(x="review_count",title="review_count")
ggplot(data=Items) +
    geom_histogram(aes(review_count))+
    labs(x="review_count",title="review_count") + xlim(1,2100)

In [37]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in numlst) {
    suppressWarnings(
        if (n != 'review_count') {
                plot(y= Items$review_count, x=Items[[n]], xlab=n, ylab='review_count',
                main=paste('review_count vs. ',n), ylim=c(0,30000))
            }
        )
    }

Review count is related to category size rank, item rating rank by category,revised star rating,star rating
corelation with description word count

In [38]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
plot(y= sqrt(Items$review_count), x=sqrt(Items$description_word_cnt), xlab="description_word_cnt", ylab='review_count',
                main=paste('review_count vs. description_word_cnt'),xlim=c(0,5000))
plot(y= sqrt(Items$review_count), x=Items$ebay_price, xlab="ebay_price", ylab='review_count',
                main=paste('review_count vs. ebay_price'),xlim=c(0,500))
plot(y= sqrt(Items$review_count), x=Items$profit, xlab="profit", ylab='review_count',
                main=paste('review_count vs. profit'),xlim=c(0,500))
plot(y= Items$review_count, x=Items$price_diff_competitor, xlab=n, ylab='review_count',
    main='review_count vs. price_diff_competitor', ylim =c(0,20000), xlim=c(-200,200))

we see correlation with ebay_price and profit

In [39]:
df <- Items %>% 
    group_by(review_count,star_rating) %>% 
    summarise(mean=mean(star_rating,na.rm=T), sd=sd(star_rating,na.rm=T),total=n()) 
#df %>% arrange(desc(total))
barplot(height=df$total, names=df$review_count)

In [40]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in strlst) {
    suppressWarnings(
        if (n != 'brand' && n != 'review_count') {
            boxplot(review_count ~ Items[[n]], data = Items, xlab=n, main=paste('review_count count vs. ',n))
            }
        )
    }

In [41]:
Items_t <- filter(Items,review_count<= 2062) # Q3
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in strlst) {
    suppressWarnings(
        if (n != 'brand' && n != 'review_count') {
            boxplot(review_count ~ Items_t[[n]], data = Items_t, xlab=n, main=paste('review_count count vs. ',n))
            }
        )
    }

review count is correlated with all sales days, popular brand and reviews cnt category

# Variable: star_rating

In [42]:
initial_EDA(Items$star_rating)
# percent of values > Q3
length(na.omit(which(Items$star_rating > quantile(Items$star_rating, probs=0.75,,na.rm=T))))/
    length(na.omit(Items$star_rating,,na.rm=T))*100
table(Items$star_rating)

In [43]:
options(repr.plot.width = 5, repr.plot.height = 5)
ggplot(data=Items) +
    geom_histogram(aes(star_rating))+
    labs(x="star_rating",title="star_rating")
ggplot(data=Items) +
    geom_histogram(aes(star_rating))+
    labs(x="star_rating",title="star_rating") + xlim(3.9,5.1)

In [44]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in numlst) {
    suppressWarnings(
        if (n != 'star_rating') {
                plot(y= Items$star_rating, x=Items[[n]], xlab=n, ylab='star_rating',
                main=paste('star_rating vs. ',n)#,xlim=c(-10000,9000)
                    )
            }
        )
    }

In [45]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
plot(y= Items$star_rating, x=Items$ebay_price, xlab='ebay_price', ylab='star_rating',
                main='star_rating vs. ebay_price',xlim=c(0,1000))
plot(y= Items$star_rating, x=Items$review_count, xlab='review_count', ylab='star_rating',
                main='star_rating vs. review_count',xlim=c(0,10000))
plot(y= Items$star_rating, x=Items$competitor_price, xlab='competitor_price', ylab='star_rating',
                main='star_rating vs. competitor_price',xlim=c(0,1000))
plot(y= Items$star_rating, x=Items$amazon_price, xlab='amazon_price', ylab='star_rating',
                main='star_rating vs. amazon_price',xlim=c(0,1000))
plot(y= Items$star_rating, x=Items$profit, xlab='profit', ylab='star_rating',
                main='star_rating vs. profit',xlim=c(0,1000))
plot(y= Items$star_rating, x=Items$profit_pct, xlab='profit_pct', ylab='star_rating',
                main='star_rating vs. profit_pct',xlim=c(-100,100))
plot(y= Items$star_rating, x=Items$price_diff_competitor, xlab='price_diff_competitor', ylab='star_rating',
                main='star_rating vs. price_diff_competitor',xlim=c(-100,100))

In [46]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in strlst) {
    suppressWarnings(
        if (n != 'brand' && n != 'star_rating') {
            boxplot(star_rating ~ Items[[n]], data = Items, xlab=n, main=paste('star_rating vs. ',n))
            }
        )
    }

In [47]:
Items_t <- filter(Items,star_rating >= 4)
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in strlst) {
    suppressWarnings(
        if (n != 'brand' && n != 'star_rating') {
            boxplot(star_rating ~ Items_t[[n]], data = Items_t, xlab=n, main=paste('star_rating vs. ',n))
            }
        )
    }

star rating is associated to category size rank,item rating rank by category, description word count,reviews count, profit, price diff competitor.

star rating is corellated to ebay price, amazon price,competitor price

# Variable: revised_star_rating

In [48]:
initial_EDA(Items$revised_star_rating)
# percent of values > Q3
length(na.omit(which(Items$revised_star_rating > quantile(Items$revised_star_rating, probs=0.75,na.rm=T))))/
    length(na.omit(Items$revised_star_rating,,na.rm=T))*100

In [49]:
library(ggplot2)
rating1 <- Items$revised_star_rating[which(Items$revised_star_rating>3)]
initial_EDA(rating1)
rating1 <- data.frame(rating1)
ggplot(data=rating1) +
    geom_histogram(aes(rating1))+
    labs(x="revised_star_rating",title="revised_star_rating category med/high")

In [50]:
library(ggplot2)
options(repr.plot.width = 5, repr.plot.height = 5)
ggplot(data=Items) +
    geom_histogram(aes(revised_star_rating))+
    labs(x="revised_star_rating",title="revised_star_rating")
ggplot(data=Items) +
    geom_histogram(aes(revised_star_rating))+
    labs(x="revised_star_rating",title="revised_star_rating")+ xlim(3,50)

In [51]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in numlst) {
    suppressWarnings(
        if (n != 'revised_star_rating') {
                plot(y= Items$revised_star_rating, x=Items[[n]], xlab=n, ylab='revised_star_rating',
                main=paste('revised_star_rating vs. ',n))#,xlim=c(-10000,9000))
            }
        )
    }

In [52]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
plot(y= Items$revised_star_rating, x=Items$ebay_price, xlab='ebay_price', ylab='revised_star_rating',
                main='revised_star_rating vs. ebay_price',xlim=c(0,1000))
plot(y= Items$revised_star_rating, x=Items$competitor_price, xlab='competitor_price', ylab='revised_star_rating',
                main='revised_star_rating vs. competitor_price',xlim=c(0,1000))
plot(y= Items$revised_star_rating, x=Items$amazon_price, xlab='amazon_price', ylab='revised_star_rating',
                main='revised_star_rating vs. amazon_price',xlim=c(0,1000))
plot(y= Items$revised_star_rating, x=Items$profit, xlab='profit', ylab='revised_star_rating',
                main='revised_star_rating vs. profit',xlim=c(0,1000))
plot(y= Items$revised_star_rating, x=Items$profit_pct, xlab='profit_pct', ylab='revised_star_rating',
                main='revised_star_rating vs. profit_pct',xlim=c(-100,100))
plot(y= Items$revised_star_rating, x=Items$price_diff_competitor, xlab='price_diff_competitor', ylab='revised_star_rating',
                main='revised_star_rating vs. price_diff_competitor',xlim=c(-100,100))

In [53]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in strlst) {
    suppressWarnings(
        if (n != 'brand' && n != 'revised_star_rating') {
            boxplot(revised_star_rating ~ Items[[n]], data = Items, xlab=n,
                    main=paste('revised_star_rating vs. ',n), outline=FALSE)
            }
        )
    }

variable revised star rating is associated with ebay price, amazon price, and star rating

correlated with rating rank by category, review count, popular brand and revies cnt category

# Variable: ebay price

In [54]:
initial_EDA(Items$ebay_price)
# percent of values higher that Q3
length(which(Items$ebay_price > quantile(Items$ebay_price, probs=0.75)))/length(Items$ebay_price)*100

length(which(Items$ebay_price > 30.56))
sum(is.na(Items$ebay_price)) # count NA

In [55]:
length(which(Items$ebay_price > 5000))

In [56]:
#require(gridExtra)
library(ggplot2)
options(repr.plot.width = 10, repr.plot.height = 5)
par(mfrow=c(1,2))
    ggplot(data=Items) +
        geom_histogram(aes(ebay_price),bins=500)+
        labs(x="ebay_price",title="ebay_price")+ xlim(0,500)
    hist(Items$ebay_price[which(Items$ebay_price<100)],main='Ebay price')


In [ ]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in numlst) {
    suppressWarnings(
        if (n != 'ebay_price') {
                plot(y= Items$ebay_price, x=Items[[n]], xlab=n, ylab='ebay_price',
                main=paste('ebay_price vs. ',n),ylim=c(0,5000))
            }
        )
    }

Variable Ebay_price is correlated with amazon price and profit

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
plot(y= Items$ebay_price, x=Items$returns_cnt, xlab='returns_cnt', ylab='ebay_price',
                main=paste('ebay_price vs. returns_cnt'),ylim=c(0,5000),xlim=c(0,10))
plot(y= Items$ebay_price, x=Items$review_count, xlab='review_count', ylab='ebay_price',
                main=paste('ebay_price vs. review_count'),ylim=c(0,5000),xlim=c(0,1000))

In [5]:
library(ggplot2)
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in strlst) {
        suppressWarnings(
            if (n != 'brand' && n != 'ebay_price') {
                boxplot(ebay_price ~ Items[[n]], data = Items, xlab=n, main=paste('ebay_price vs. ',n), outline=FALSE)
            }
        )
    }

Ebay price is correlated with loction,popular brand,reviews count category,item sells rank by category,number of sales per item, sale days (xmas,cyber monday,etc), condition

# Variable: Competitor price

In [7]:
initial_EDA(Items$competitor_price)

In [8]:
#require(gridExtra)
library(ggplot2)
options(repr.plot.width = 5, repr.plot.height = 5)
#par(mfrow=c(1,2))
    ggplot(data=Items) +
        geom_histogram(aes(competitor_price),bins=500)+
        labs(x="competitor_price",title="competitor_price") + xlim(0,500)
    hist(Items$competitor_price[which(Items$competitor_price<100)],main='competitor_price')


In [9]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in numlst) {
    suppressWarnings(
        if (n != 'competitor_price') {
                plot(y= Items$competitor_price, x=Items[[n]], xlab=n, ylab='competitor_price',
                main=paste('competitor_price vs. ',n),ylim=c(0,5000))
            }
        )
    }

In [10]:
library(ggplot2)
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in strlst) {
    suppressWarnings(
        if (n != 'brand' && n != 'competitor_price') {
            boxplot(competitor_price ~ Items[[n]], data = Items, xlab=n, main=paste('competitor_price vs. ',n)
                    , outline=FALSE)
            }
        )
    }

variable competitor price is associated with black_friday sale,labor day,popular brand

variable competitor price is correlated with loction,condition,eBay price,amazon price and profit

In [11]:
initial_EDA(Items$amazon_price)

# Variable: profit

In [12]:
initial_EDA(Items$profit)

In [13]:
library(ggplot2)
options(repr.plot.width = 5, repr.plot.height = 5)
    ggplot(data=Items) +
        geom_histogram(aes(profit),bins=500)+
        labs(x="profit",title="profit") + xlim(0,500)
    hist(Items$profit[which(Items$profit<50)],main='profit')

In [14]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in numlst) {
    suppressWarnings(
        if (n != 'profit') {
                plot(y= Items$profit, x=Items[[n]], xlab=n, ylab='profit',
                main=paste('profit vs. ',n), xlim=c(-1000,4000))
            }
        )
    }

In [15]:
library(ggplot2)
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in strlst) {
    suppressWarnings(
        if (n != 'brand' && n != 'profit') {
            boxplot(profit ~ Items[[n]], data = Items, xlab=n, main=paste('profit vs. ',n),
                   outline=FALSE)
            }
        )
    }

variable profit is correlated with eBay price, competitor price and amazon price, condition, location, sale days: black friday,labor day,4th of july, item_sell_rank by category

# Variable: Profit percent

In [16]:
initial_EDA(Items$profit_pct)

In [17]:
library(ggplot2)
options(repr.plot.width = 5, repr.plot.height = 5)
#par(mfrow=c(1,2))
    ggplot(data=Items) +
        geom_histogram(aes(profit_pct),bins=500)+
        labs(x="profit_pct",title="profit_pct") + xlim(-100,100)
    hist(Items$profit_pct[which(Items$profit_pct>-50)],main='profit_pct')

In [18]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in numlst) {
    suppressWarnings(
        if (n != 'profit') {
                plot(y= Items$profit_pct, x=Items[[n]], xlab=n, ylab='profit_pct',
                main=paste('profit_pct vs. ',n)
                     , ylim=c(-1000,100)
                    )
            }
        )
    }

In [19]:
library(ggplot2)
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in strlst) {
    suppressWarnings(
        if (n != 'brand' && n != 'profit_pct') {
            boxplot(profit_pct ~ Items[[n]], data = Items, xlab=n, main=paste('profit_pct vs. ',n),
                    outline=FALSE)
            }
        )
    }

profit percent is associated with sale days:colombus_day,new_year,xmas,black_friday,president_day,cyber_monday, condition and location

# variable price_diff competitor

In [20]:
initial_EDA(Items$price_diff_competitor)
filter(Items,price_diff_competitor== -999) %>% summarize(n()) # count how many values are -999, i.e. NA
summary(Items$price_diff_competitor[Items$price_diff_competitor > -999])
filter(Items,price_diff_competitor== 0) %>% summarize(n()) # count how many values are 0

In [21]:
ggplot(data=Items) +
    geom_histogram(aes(price_diff_competitor),bins=50)+
        labs(x="price_diff_competitor",title="price_diff_competitor")# + xlim(-100,100)
hist(Items$price_diff_competitor,main='price_diff_competitor')
ggplot(data=Items) +
   geom_density(aes(log(price_diff_competitor+100))) + xlim(4,5) # add constant 100 (>min(price_diff_competitor)) to avoid negative data

In [22]:
cor <- Items$price_diff_competitor[Items$price_diff_competitor >-999]
summary(log(cor+100))

In [23]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in numlst) {
    suppressWarnings(
        if (n != 'price_diff_competitor') {
                plot(y= Items$price_diff_competitor, x=Items[[n]], xlab=n, ylab='price_diff_competitor',
                main=paste('price_diff_competitor vs. ',n))
            }
        )
    }

In [24]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in numlst) {
    suppressWarnings(
        if (n != 'price_diff_competitor') {
                plot(y= Items$price_diff_competitor, x=Items[[n]], xlab=n, ylab='price_diff_competitor',
                main=paste('price_diff_competitor vs. ',n), ylim=c(-100,100))
            }
        )
    }

In [25]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(5,5,5,5),mfrow=c(1,4))
plot(y= Items$price_diff_competitor, x=Items$ebay_price, xlab='ebay_price', ylab='price_diff_competitor',
                main='price_diff_competitor vs. ebay_price',xlim=c(0,1000),ylim=c(-100,100))
plot(y= Items$price_diff_competitor, x=Items$review_count, xlab='review_count', ylab='price_diff_competitor',
                main='price_diff_competitor vs. review_count',xlim=c(0,10000),ylim=c(-100,100))
plot(y= Items$price_diff_competitor, x=Items$competitor_price, xlab='competitor_price', ylab='price_diff_competitor',
                main='price_diff_competitor vs. competitor_price',xlim=c(0,1000),ylim=c(-100,100))
plot(y= Items$price_diff_competitor, x=Items$amazon_price, xlab='amazon_price', ylab='price_diff_competitor',
                main='price_diff_competitor vs. amazon_price',xlim=c(0,1000),ylim=c(-100,100))
plot(y= Items$price_diff_competitor, x=Items$profit, xlab='profit', ylab='price_diff_competitor',
                main='price_diff_competitor vs. profit',xlim=c(0,1000),ylim=c(-100,100))

In [26]:
library(ggplot2)
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in strlst) {
    suppressWarnings(
        if (n != 'brand' && n != 'price_diff_competitor') {
            boxplot(price_diff_competitor ~ Items[[n]], data = Items, xlab=n, main=paste('price_diff_competitor vs. ',n),
                    outline=FALSE)
            }
        )
    }

variable price_diff_competitor is associated with sale days

# Variable: item_rating_rank_by_category

In [27]:
initial_EDA(Items$item_rating_rank_by_category)

In [28]:
library(ggplot2)
options(repr.plot.width = 8, repr.plot.height = 8)
#par(mfrow=c(1,3))
    hist(Items$item_rating_rank_by_category,main='item_rating_rank_by_category')
    ggplot(data=Items) +
        geom_histogram(aes(item_rating_rank_by_category),bins=100)+
        labs(x="item_rating_rank_by_category",title="item_rating_rank_by_category")# + xlim(0,500)
    hist(sqrt(Items$item_rating_rank_by_category),main='item_rating_rank_by_category',)

In [29]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in numlst) {
    suppressWarnings(
        if (n != 'item_rating_rank_by_category') {
                plot(y= Items$item_rating_rank_by_category, x=Items[[n]], xlab=n, ylab='item_rating_rank_by_category',
                main=paste('item_rating_rank_by_category vs. ',n))
            }
        )
    }

In [30]:
options(repr.plot.width = 10, repr.plot.height = 5)
par(mar=c(3,3,3,3),mfrow=c(1,2))
plot(y= log(Items$item_rating_rank_by_category), x=log(Items$review_count), xlab='review_count', ylab='item_rating_rank_by_category',
                main=paste('item_rating_rank_by_category vs. review_count'))
plot(y= sqrt(Items$item_rating_rank_by_category), x=(Items$revised_star_rating), xlab='revised_star_rating', ylab='item_rating_rank_by_category',
                main=paste('item_rating_rank_by_category vs. revised_star_rating'))

In [31]:
library(ggplot2)
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,5,3,3),mfrow=c(1,4))
for (n in strlst) {
    suppressWarnings(
        if (n != 'brand' && n != 'item_rating_rank_by_category') {
            boxplot(item_rating_rank_by_category ~ Items[[n]], data = Items, xlab=n, main=paste('item_rating_rank_by_category vs. ',n)
             ,outline=FALSE)
            }
        )
    }

In [32]:
boxplot( Items$item_rating_rank_by_category ~ Items$reviews_cnt_category,
        xlab=Items$reviews_cnt_category, main=paste('item_rating_rank_by_category vs. reviews_cnt_category'))

rating rank by category is correlated with reviews count and revised star rating

association with reviews count category

# Variable: category_size_rank

In [33]:
initial_EDA(Items$category_size_rank)
table(Items$category_size_rank)

In [34]:
library(ggplot2)
options(repr.plot.width = 8, repr.plot.height = 8)
#par(mfrow=c(1,3))
    hist(Items$category_size_rank,main='category_size_rank')

In [35]:
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in numlst) {
    suppressWarnings(
        if (n != 'category_size_rank') {
                plot(y= Items$category_size_rank, x=Items[[n]], xlab=n, ylab='category_size_rank',
                main=paste('category_size_rank vs. ',n))
            }
        )
    }

In [36]:
options(repr.plot.width = 7, repr.plot.height = 7)
plot(y= Items$category_size_rank, x=sqrt(Items$ebay_price), xlab='ebay_price', ylab='category_size_rank',
                main=paste('category_size_rank vs. ebay_price'))
plot(y= Items$category_size_rank, x=sqrt(Items$item_rating_rank_by_category), xlab='item_rating_rank_by_category', ylab='category_size_rank',
                main=paste('category_size_rank vs. item_rating_rank_by_category'))

In [38]:
library(ggplot2)
options(repr.plot.width = 20, repr.plot.height = 4)
par(mar=c(3,3,3,3),mfrow=c(1,4))
for (n in strlst) {
    suppressWarnings(
        if (n != 'brand' && n != 'category_size_rank') {
            boxplot(category_size_rank ~ Items[[n]], data = Items, xlab=n, main=paste('category_size_rank vs. ',n)
            ,outline=FALSE)
            }
        )
    }

correlation to item_rating_rank_by_category